# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto baquerizo moreno,-0.9000,-89.6000,299.70,74,75,4.12,EC,1696269152
1,1,wenchang,30.9040,118.4887,293.07,80,98,1.56,CN,1696269152
2,2,adamstown,-25.0660,-130.1015,293.41,73,22,10.89,PN,1696269142
3,3,ushuaia,-54.8000,-68.3000,280.94,45,0,4.12,AR,1696269070
4,4,port hedland,-20.3167,118.5667,294.18,94,8,2.06,AU,1696268854


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(x='Lng',
                            y='Lat',
                            geo=True,
                            tiles='OSM',
                            frame_width = 700,
                            frame_height = 500,
                            size = 'Humidity',
                            scale = .50,
                            color = "City")

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# changed to celcius
city_data_df["Max Temp"] = city_data_df["Max Temp"] - 273.15
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto baquerizo moreno,-0.9000,-89.6000,26.55,74,75,4.12,EC,1696269152
1,1,wenchang,30.9040,118.4887,19.92,80,98,1.56,CN,1696269152
2,2,adamstown,-25.0660,-130.1015,20.26,73,22,10.89,PN,1696269142
3,3,ushuaia,-54.8000,-68.3000,7.79,45,0,4.12,AR,1696269070
4,4,port hedland,-20.3167,118.5667,21.03,94,8,2.06,AU,1696268854


In [35]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 28)]

# Drop any rows with null values
filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto baquerizo moreno,-0.9000,-89.6000,26.55,74,75,4.12,EC,1696269152
2,2,adamstown,-25.0660,-130.1015,20.26,73,22,10.89,PN,1696269142
4,4,port hedland,-20.3167,118.5667,21.03,94,8,2.06,AU,1696268854
5,5,jamestown,42.0970,-79.2353,26.68,53,0,1.54,US,1696268973
6,6,linda,39.1277,-121.5508,23.10,55,0,1.54,US,1696269153


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto baquerizo moreno,EC,-0.9000,-89.6000,74,
2,adamstown,PN,-25.0660,-130.1015,73,
4,port hedland,AU,-20.3167,118.5667,94,
5,jamestown,US,42.0970,-79.2353,53,
6,linda,US,39.1277,-121.5508,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":1,
    "radius":radius,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto baquerizo moreno - nearest hotel: Dory's House
adamstown - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
jamestown - nearest hotel: DoubleTree Jamestown
linda - nearest hotel: Comfort Suites Marysville-Yuba City
shahpur - nearest hotel: Hotel Jaya Lodging & Boarding
queenstown - nearest hotel: Queens Hotel
port mathurin - nearest hotel: Escale Vacances
rundu - nearest hotel: Kavango River Lodge
bethel - nearest hotel: Hampton Inn Danbury
antsiranana - nearest hotel: Central
ravar - nearest hotel: زائر سرا راور
hilo - nearest hotel: Dolphin Bay Hotel
ingeniero luis a. huergo - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
chongwe - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
avarua - nearest hotel: Paradise Inn
goure - ne

mata-utu - nearest hotel: Gîte Oceania
olavarria - nearest hotel: Santa Rosa Hotel
mwene-ditu - nearest hotel: Hôtel KAMAS
dingli - nearest hotel: Maple farm B&B
lospalos - nearest hotel: Hotel Roberto Carlos
nueve de julio - nearest hotel: Hotel Madrid
albertina - nearest hotel: Hotel Parque das Primaveras
ankazobe - nearest hotel: Promenade Annexe
mombasa - nearest hotel: Hotel Sapphire
bima - nearest hotel: No hotel found
gobabis - nearest hotel: Kalahari Convention Centre Hotel
saint-francois - nearest hotel: Chez Lily
figuig (centre) - nearest hotel: No hotel found
pemangkat - nearest hotel: Hotel Kelapa Gading
labasa - nearest hotel: Grand Eastern Hotel
oxapampa - nearest hotel: Yanachaga
sangin - nearest hotel: No hotel found
broomfield - nearest hotel: Comfort Suites Broomfield-Boulder/Interlocken
sao jose da coroa grande - nearest hotel: Hotel do Sol
mongagua - nearest hotel: No hotel found
baise city - nearest hotel: 梦之源大酒店
kapa'a - nearest hotel: Pono Kai Resort
albox - near

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto baquerizo moreno,EC,-0.9000,-89.6000,74,Dory's House
2,adamstown,PN,-25.0660,-130.1015,73,No hotel found
4,port hedland,AU,-20.3167,118.5667,94,The Esplanade Hotel
5,jamestown,US,42.0970,-79.2353,53,DoubleTree Jamestown
6,linda,US,39.1277,-121.5508,55,Comfort Suites Marysville-Yuba City
...,...,...,...,...,...,...
545,sao jose da coroa grande,BR,-8.8978,-35.1478,64,Hotel do Sol
546,mongagua,BR,-24.0931,-46.6208,81,No hotel found
552,baise city,CN,23.8997,106.6133,86,梦之源大酒店
553,kapa'a,US,22.0752,-159.3190,89,Pono Kai Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(x='Lng',
                            y='Lat',
                            geo=True,
                            tiles='OSM',
                            frame_width = 700,
                            frame_height = 500,
                            size = 'Humidity',
                            scale = 1.0,
                            color = "City",
                            hover_cols = ['Hotel Name'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)